In [10]:
from typing import Tuple
from typing import List

class AdjacencyTuples(object):
  
  def __init__(self):
    self._adjacencyMap = {}
    
  def Add(self, key: str, value: Tuple[str, str]):
    self._adjacencyMap[key] = value
    
  def Next(self, current: str, direction: str):
    direction_index = 0 if direction == 'L' else 1
    return self._adjacencyMap[current][direction_index]
  
  def Parse(lines: List[str]):
    adjacency_tuples = AdjacencyTuples()
    for line in lines:
      key_str, value_str = line.split('=')
      key_str = key_str.strip()
      value_str = value_str.strip()[1:-1].split(',')
      left = value_str[0].strip()
      right = value_str[1].strip()
      adjacency_tuples.Add(key_str, (left, right))
    return adjacency_tuples
      

# open text file
def ReadFile(filename: str):
  with open(filename, 'r') as f:
    lines = f.readlines()
  return lines

# parse text file
def ParseFile(lines: List[str]):
  directions = lines[0].strip()
  adjacency_tuples = AdjacencyTuples.Parse(lines[2:])
  return directions, adjacency_tuples

def SolvePartOne(filename: str):
  lines = ReadFile(filename)
  directions, adjacency_tuples = ParseFile(lines)
  current = 'AAA'
  idx = 0
  steps = 0
  while current != 'ZZZ':
    current = adjacency_tuples.Next(current, directions[idx])
    steps += 1
    idx = steps % len(directions)
  return steps

assert SolvePartOne('sample.txt') == 2
assert SolvePartOne('sample2.txt') == 6
part_one_solution = SolvePartOne('input.txt')
print('Part One Solution:', part_one_solution)
assert part_one_solution == 17621

Part One Solution: 17621
